In [6]:
import numpy as np
import pandas as pd
from Bio import SeqIO
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from Bio.SeqUtils.ProtParam import ProtParamData
from quantiprot.metrics.aaindex import get_aa2volume
from sklearn.model_selection import train_test_split



In [7]:
WINDOW_SIZE = 9
NUM_OF_FEATURES = 6

In [8]:
def open_acid_file() -> pd.DataFrame:
    fasta_sequences = SeqIO.parse(open("iedb_linear_epitopes.fasta"),'fasta')
    names = [fasta_seq.id for fasta_seq in fasta_sequences]
    sequences = [str(fasta_seq.seq) for fasta_seq in fasta_sequences]
    return pd.DataFrame(data={"name": names, "sequence": sequences})

In [9]:
def split_to_windows(protein):
    # TODO
    return None

In [10]:
def train_main(train_data):
    for protein in train_data:
        amino_windows = split_to_windows(protein)
        for amino_window in amino_windows:
            input_matrix = compute_matrix(amino_window)
            label = int(amino_window[4].isupper())

In [11]:
def compute_matrix(amino_window: str) -> np.ndarray:
    input_matrix = np.zeros((WINDOW_SIZE, NUM_OF_FEATURES))
    for amino_acid, i in enumerate(amino_window):
        analysed_acid = ProteinAnalysis(amino_acid.upper())
        volume = get_aa2volume(analysed_acid)
        hydrophobicity = analysed_acid.protein_scale(window=WINDOW_SIZE, param_dict=ProtParamData.kd)
        polarity = compute_polarity(amino_acid)
        rsa = compute_ss(amino_acid)
        ss = compute_s(amino_acid)
        amino_type = compute_type(amino_acid)
        input_matrix[i] = np.asarray([amino_acid, volume, hydrophobicity, polarity, rsa, ss, amino_type])
    return input_matrix

In [12]:
def main():
    proteins = open_acid_file()
    train, test = train_test_split(proteins, test_size=0.3)
    model = train_main(train)
    test_model(model, test)
    # TODO: show performance somehow
